In [3]:
import requests
import logging

# Настройка базового логирования для вывода ошибок
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def fetch_html_content(url: str) -> str | None:
    """
    Принимает URL, отправляет GET-запрос и возвращает HTML-код страницы.

    :param url: Адрес страницы.
    :return: Строка с HTML-кодом или None в случае ошибки.
    """
    try:
        logging.info(f"Попытка загрузки URL: {url}")
        
        # Отправляем GET-запрос с таймаутом
        response = requests.get(url, timeout=15)
        
        # Проверяем код состояния (200 OK, 301/302 редирект и т.д.)
        # Если статус-код >= 400 (ошибка клиента/сервера), будет выброшено исключение
        response.raise_for_status() 
        
        # Устанавливаем корректную кодировку для русскоязычных сайтов
        response.encoding = response.apparent_encoding
        
        logging.info("SUCCESS: Страница успешно загружена.")
        return response.text
    
    except requests.exceptions.HTTPError as e:
        logging.error(f"Ошибка HTTP (Код: {response.status_code}): {e}")
        return None
    except requests.exceptions.ConnectionError as e:
        logging.error(f"Ошибка подключения: Не удалось установить соединение с {url}")
        return None
    except requests.exceptions.Timeout:
        logging.error(f"Таймаут: Превышено время ожидания ответа от {url}")
        return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Непредвиденная ошибка при запросе: {e}")
        return None

# --- ПРИМЕР ИСПОЛЬЗОВАНИЯ ---
test_url = "https://priem.stankin.ru/obshchezhitie/"

html_code = fetch_html_content(test_url)

if html_code:
    print("\n--- ПЕРВЫЕ 500 СИМВОЛОВ HTML-КОДА ---")
    print(html_code[:])
    print("...")

INFO: Попытка загрузки URL: https://priem.stankin.ru/obshchezhitie/
INFO: SUCCESS: Страница успешно загружена.



--- ПЕРВЫЕ 500 СИМВОЛОВ HTML-КОДА ---
<!DOCTYPE html>
<html xml:lang="ru" lang="ru" class="">
<head>
	<!-- Yandex.Metrika counter -->
<script data-skip-moving="true">(function(m,e,t,r,i,k,a){m[i]=m[i]||function(){(m[i].a=m[i].a||[]).push(arguments)};
					m[i].l=1*new Date();
					for (var j = 0; j < document.scripts.length; j++) {if (document.scripts[j].src === r) { return; }}
					k=e.createElement(t),a=e.getElementsByTagName(t)[0],k.async=1,k.src=r,a.parentNode.insertBefore(k,a)})
					(window, document, "script", "https://mc.yandex.ru/metrika/tag.js", "ym");
					ym('104165891', "init", {
						clickmap:true,
						trackLinks:true,
						accurateTrackBounce:true,
						webvisor:true,
						trackHash:true,
						ecommerce:"dataLayer"
				   });</script>
<!-- /Yandex.Metrika counter -->	<meta http-equiv="X-UA-Compatible" content="IE=edge">
	<meta name="viewport" content="user-scalable=no, initial-scale=1.0, maximum-scale=1.0, width=device-width">
	<meta name="HandheldFriendly" conte

In [6]:
import requests
import logging
import re
from bs4 import BeautifulSoup, Comment
from typing import Optional

# Настройка базового логирования для вывода ошибок
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def fetch_html_content(url: str) -> Optional[str]:
    # (Функция fetch_html_content остается без изменений)
    try:
        logging.info(f"Попытка загрузки URL: {url}")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, timeout=15, headers=headers)
        response.raise_for_status() 
        response.encoding = response.apparent_encoding
        logging.info("SUCCESS: Страница успешно загружена.")
        return response.text
    
    except requests.exceptions.HTTPError as e:
        status_code = getattr(response, 'status_code', 'N/A')
        logging.error(f"Ошибка HTTP (Код: {status_code}): {e}")
        return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка запроса к {url}: {e}")
        return None


def extract_pure_text(html_content: str) -> str:
    """
    Агрессивно очищает HTML-код, удаляя сквозные блоки и извлекая
    структурированный чистый текст с сохранением переносов строк.

    :param html_content: Строка с полным HTML-кодом страницы.
    :return: Очищенный текст в виде строки.
    """
    soup = BeautifulSoup(html_content, 'html.parser')

    # 1. Удаление служебных, нетекстовых тегов и комментариев
    for element in soup(['script', 'style', 'noscript', 'iframe', 'meta', 'link', 'br']):
        element.decompose()
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()

    # 2. Удаление сквозных и нерелевантных блоков (навигация, футер)
    elements_to_remove = [
        soup.find('div', class_='block-0-menu-16'),
        soup.find('nav', id='menu'),
        soup.find('div', id='n'),
        soup.find('header', class_='landing-header'),
        soup.find('div', class_='landing-footer'), # Удаляем футер
        soup.find('style', type='text/css'),
    ]

    for element in elements_to_remove:
        if element:
            element.decompose()
            
    # 3. Извлечение текста из основного контейнера
    main_content_tag = soup.find('div', class_='landing-main')
    if not main_content_tag:
        main_content_tag = soup.body if soup.body else soup

    # 4. Преобразование HTML в структурированный текст

    # Теги, которые должны быть заменены на двойной перенос строки (блочные элементы)
    for tag_name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'section', 'li', 'hr']:
        for tag in main_content_tag.find_all(tag_name):
            # Вставляем перенос перед элементом
            if tag.previous_sibling and tag.previous_sibling.name != tag_name:
                tag.insert_before('\n') 
            # Вставляем перенос после элемента
            tag.append('\n')

    # Извлекаем текст
    pure_text = main_content_tag.get_text(separator=' ', strip=True)

    # 5. Финальная чистка текста
    # Заменяем несколько переносов строк/пробелов на один двойной перенос строки для абзацев
    pure_text = re.sub(r'[\s]{2,}', '\n\n', pure_text)
    pure_text = pure_text.strip()
    
    return pure_text

# --- ПРИМЕР ИСПОЛЬЗОВАНИЯ ---
test_url = "https://priem.stankin.ru/bakalavriatispetsialitet/formaprovedeniyaekzamenov/"

html_code = fetch_html_content(test_url)

if html_code:
    pure_text = extract_pure_text(html_code)
    print("\n--- ОЧИЩЕННЫЙ, СТРУКТУРИРОВАННЫЙ ТЕКСТ (Готов для LLM) ---")
    print(pure_text)
    print("\n--- КОЛИЧЕСТВО СИМВОЛОВ В ЧИСТОМ ТЕКСТЕ: {} ---".format(len(pure_text)))

INFO: Попытка загрузки URL: https://priem.stankin.ru/bakalavriatispetsialitet/formaprovedeniyaekzamenov/
INFO: SUCCESS: Страница успешно загружена.



--- ОЧИЩЕННЫЙ, СТРУКТУРИРОВАННЫЙ ТЕКСТ (Готов для LLM) ---
Форма проведения экзаменов Здесь указанная информация представлена в сжатом и понятном формате. С полной информацией можно ознакомиться здесь . Вступительные испытания могут сдавать: На базе среднего общего образования: инвалиды (в том числе дети-инвалиды) (вне зависимости от наличия ЕГЭ); лица, указанные в части 5.1 статьи 71 Федерального закона № 273-ФЗ (вне зависимости вида мест, вне зависимости от наличия ЕГЭ); лица, имеющие право на прием на места в пределах отдельной квоты по результатам ЕГЭ или вступительных испытаний в соответствии с частью 5.2 статьи 71 Федерального закона № 273-ФЗ (вне зависимости от наличия ЕГЭ). иностранные граждане (по тем предметам, по которым поступающий не сдавал ЕГЭ); поступающие, которые имеют документ о среднем общем образовании, полученный в иностранной организации (по тем предметам, по которым поступающий не сдавал ЕГЭ в текущем календарном году); лица, попадающие под особенности приёма на

In [11]:
import requests
import logging
import re
import io
import pytesseract
from PIL import Image
from typing import Optional, Tuple, List
from bs4 import BeautifulSoup, Comment
import pdfplumber
# from pytesseract import pytesseract # Раскомментировать, если нужно задать путь

# ... (fetch_html_content и extract_pdf_links_and_clean остаются прежними, для краткости опущены) ...

def extract_text_from_pdf_with_ocr(pdf_url: str) -> Optional[str]:
    """
    Скачивает PDF и пытается извлечь текст. Если не удается,
    пытается использовать OCR (Tesseract).
    """
    try:
        logging.info(f"Скачивание и парсинг PDF: {pdf_url}")
        
        pdf_response = requests.get(pdf_url, timeout=30)
        pdf_response.raise_for_status()
        pdf_file = io.BytesIO(pdf_response.content)
        
        full_text = []

        # Попытка 1: Обычное извлечение текста с помощью pdfplumber
        try:
            with pdfplumber.open(pdf_file) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        full_text.append(text)

            if len(full_text) > 0:
                logging.info(f"SUCCESS: Извлечено {len(full_text)} страниц (текстовый слой).")
                final_text = "\n\n".join(full_text)
                return re.sub(r'[\s]{2,}', '\n', final_text).strip()
            
            logging.warning("WARNING: Текстовый слой в PDF отсутствует. Переход к OCR.")

        except Exception as e:
            logging.warning(f"WARNING: Ошибка pdfplumber ({e}). Переход к OCR.")
            pdf_file.seek(0) # Перезагружаем файл для Tesseract

        # Попытка 2: OCR (Распознавание изображений)
        try:
            # Установите путь к исполняемому файлу Tesseract, если он не прописан в PATH:
            # pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 
            
            # Tesseract умеет работать с PDF-файлами, но лучше преобразовать PDF в изображение
            # или использовать более прямолинейный метод, если pdfplumber не сработал.
            
            # Для простоты и надёжности, если pdfplumber не дал текста, мы считаем PDF как изображение.
            # Внимание: для конвертации PDF в изображения через Python нужен дополнительный инструмент (Ghostscript или Poppler). 
            # Поскольку это усложняет установку, мы используем более простой подход:
            
            # Tesseract может обрабатывать PDF напрямую (если у него есть Ghostscript, который он использует для конвертации).
            # Мы вызываем image_to_string с указанием языка (русский).
            
            # Примечание: Для надёжной работы с PDF-сканами через Tesseract требуется установка Ghostscript.
            # Если код упадет здесь, то проблема в отсутствии вспомогательных инструментов.
            
            text_ocr = pytesseract.image_to_string(pdf_file, lang='rus')
            
            if text_ocr:
                logging.info("SUCCESS: Текст извлечен с помощью OCR (Tesseract).")
                return re.sub(r'[\s]{2,}', '\n', text_ocr).strip()
            
            logging.error("ERROR: OCR не смог извлечь текст.")
            return None

        except pytesseract.TesseractNotFoundError:
            logging.error("FATAL: Tesseract OCR не найден. Установите Tesseract и убедитесь, что он доступен в PATH.")
            return None
        except Exception as e:
            logging.error(f"ERROR: Ошибка OCR (Tesseract) при парсинге PDF: {e}")
            return None

    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка при скачивании PDF {pdf_url}: {e}")
        return None
    except Exception as e:
        logging.error(f"Непредвиденная ошибка при обработке PDF {pdf_url}: {e}")
        return None

# --- ИНТЕГРИРОВАННЫЙ БЛОК КОДА ---

def parse_stankin_page(url: str):
    html_code = fetch_html_content(url)

    if not html_code:
        return

    # Шаг 1: Извлекаем чистый текст и список PDF-ссылок
    pure_text, pdf_links = extract_pdf_links_and_clean(html_code)
    
    print("\n" + "="*50)
    print("🤖 РЕЗУЛЬТАТ ПАРСИНГА СТРАНИЦЫ")
    print("="*50)
    
    print("\n--- 1. ОЧИЩЕННЫЙ, СТРУКТУРИРОВАННЫЙ ТЕКСТ ---")
    print(pure_text)
    
    print("\n--- 2. НАЙДЕННЫЕ PDF-ДОКУМЕНТЫ (ССЫЛКИ И OCR) ---")
    if pdf_links:
        for title, url in pdf_links:
            print(f"\n- {title}: {url}")
            
            # Шаг 2: Парсинг каждого найденного PDF (с поддержкой OCR)
            pdf_content = extract_text_from_pdf_with_ocr(url)
            
            if pdf_content:
                print(f"   [СОДЕРЖИМОЕ PDF: {title[:30]}...]")
                # Печатаем только первые 500 символов для читаемости
                print(pdf_content[:500] + "\n   [...полный текст PDF урезан для отображения]")
            else:
                print("   [Не удалось извлечь текст из PDF даже с помощью OCR]")
    else:
        print("PDF-ссылки на странице не найдены.")

# --- ЗАПУСК ---
test_url = "https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/"
parse_stankin_page(test_url)

INFO: Попытка загрузки URL: https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/
INFO: SUCCESS: Страница успешно загружена.



🤖 РЕЗУЛЬТАТ ПАРСИНГА СТРАНИЦЫ

--- 1. ОЧИЩЕННЫЙ, СТРУКТУРИРОВАННЫЙ ТЕКСТ ---
Программы обучения бакалавриата, специалитета 01.03.04 Прикладная математика Форма обучения: очная Стоимость обучения для граждан РФ: 164 000 руб/сем Стоимость обучения для иностранных граждан: 172 500 руб/сем Предметы: Р (min 40 баллов) + М (min 40 баллов) + И (min 44 балла)/Ф (min 39 баллов) Количество мест: Бюджетные (с учетом квот) Платные для граждан РФ Платные для иностранных граждан — 20 5 Отдельная квота: — мест Особая квота: — мест Целевая квота: — мест Проходные баллы: 2025 2024 2023 2022 2021 — — — — — Подробнее о направлении 09.03.01 Информатика и вычислительная техника Форма обучения: очная Стоимость обучения для граждан РФ: 182 100 руб/сем Стоимость обучения для иностранных граждан: 190 000 руб/сем Предметы: Р (min 40 баллов) + М (min 40 баллов) + И (min 44 балла) Количество мест: Бюджетные (с учетом квот) Платные для граждан РФ Платные для иностранных граждан 90 30 15 Отдельная квота: 9 мест Ос

In [13]:
import requests
import logging
import re
import pandas as pd
from io import StringIO
from typing import Optional, Tuple, List
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin # Добавил для универсальности краулера

# Настройка базового логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# =========================================================
# 1. ЗАГРУЗКА HTML-КОДА
# =========================================================

def fetch_html_content(url: str) -> Optional[str]:
    """
    Принимает URL, отправляет GET-запрос и возвращает HTML-код страницы.
    """
    try:
        logging.info(f"Попытка загрузки URL: {url}")
        
        # Рекомендуется использовать User-Agent для избежания блокировок
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(url, timeout=15, headers=headers)
        response.raise_for_status()
        
        response.encoding = response.apparent_encoding
        
        logging.info("SUCCESS: Страница успешно загружена.")
        return response.text
    
    except requests.exceptions.HTTPError as e:
        status_code = getattr(response, 'status_code', 'N/A')
        logging.error(f"Ошибка HTTP (Код: {status_code}): {e}")
        return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка запроса к {url}: {e}")
        return None

# =========================================================
# 2. ОЧИСТКА И ПАРСИНГ (HTML + ТАБЛИЦЫ)
# =========================================================

def clean_html_content_and_extract_links(html_content: str, url: str) -> Tuple[str, List[Tuple[str, str]]]:
    """
    Агрессивно очищает HTML: удаляет служебные теги,
    конвертирует таблицы в Markdown и извлекает PDF-ссылки.
    """
    soup = BeautifulSoup(html_content, 'lxml')
    pdf_links = []
    
    # 1. Удаление служебных, нетекстовых тегов, комментариев и сквозных блоков
    REMOVAL_TAGS = ['script', 'style', 'nav', 'footer', 'header', 'form', 'aside', 'iframe', 'noscript', 'meta', 'link', 'br']
    for tag in REMOVAL_TAGS:
        for element in soup.find_all(tag):
            element.decompose()
            
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()
        
    elements_to_remove = [
        soup.find('div', class_='block-0-menu-16'),
        soup.find('nav', id='menu'),
        soup.find('div', id='n'),
        soup.find('header', class_='landing-header'),
        soup.find('div', class_='landing-footer'),
        soup.find('style', type='text/css'),
    ]

    for element in elements_to_remove:
        if element:
            element.decompose()
            
    # 2. Обработка Таблиц (Преобразование в Markdown)
    tables = soup.find_all('table')
    if tables:
        logging.info(f"Обработка {len(tables)} таблиц на странице {url}...")
        for i, table in enumerate(tables):
            try:
                # Используем StringIO, чтобы избежать FutureWarning в pandas
                df = pd.read_html(StringIO(str(table)))[0]
                
                markdown_table = "\n\n--- НАЧАЛО ТАБЛИЦЫ ---\n"
                markdown_table += df.to_markdown(index=False)
                markdown_table += "\n--- КОНЕЦ ТАБЛИЦЫ ---\n\n"
                
                # Заменяем тег <table> на его Markdown представление
                # Это гарантирует, что структурированный текст будет извлечен на следующем шаге
                table.replace_with(BeautifulSoup(markdown_table, 'html.parser'))
                
            except Exception as e:
                logging.warning(f"Ошибка парсинга таблицы {i+1} на {url}: {e}. Таблица удалена.")
                table.decompose() # Удаляем проблемную таблицу

    # 3. Извлечение PDF-ссылок (только ссылки)
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.lower().endswith('.pdf'):
            title = link.text.strip() or f"PDF-{len(pdf_links) + 1}"
            absolute_url = urljoin(url, href)
            pdf_links.append((title, absolute_url))
            # Удаляем PDF-ссылку из HTML, чтобы ее текст не попал в pure_text
            link.decompose()
            
    # 4. Извлечение текста из основного контейнера
    main_content_tag = soup.find('div', class_='landing-main')
    if not main_content_tag:
        main_content_tag = soup.body if soup.body else soup

    # Теги, которые должны быть заменены на перенос строки (блочные элементы)
    for tag_name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'section', 'li', 'hr']:
        for tag in main_content_tag.find_all(tag_name):
             # Вставляем перенос после элемента
            tag.append('\n')

    # Извлекаем текст
    pure_text = main_content_tag.get_text(separator=' ', strip=True)

    # 5. Финальная чистка текста
    # Заменяем несколько переносов строк/пробелов на двойной перенос строки для абзацев
    pure_text = re.sub(r'[\s]{2,}', '\n\n', pure_text)
    
    return pure_text.strip(), pdf_links


# =========================================================
# 3. ФИНАЛЬНЫЙ ЗАПУСК
# =========================================================

def parse_stankin_page(url: str):
    """
    Основная функция для парсинга страницы.
    """
    html_code = fetch_html_content(url)

    if not html_code:
        logging.error("Парсинг остановлен, не удалось загрузить HTML.")
        return

    # Шаг 1: Извлекаем чистый текст и список PDF-ссылок
    pure_text, pdf_links = clean_html_content_and_extract_links(html_code, url)
    
    print("\n" + "="*70)
    print(f"🤖 РЕЗУЛЬТАТ ПАРСИНГА СТРАНИЦЫ: {url}")
    print("="*70)
    
    print("\n--- 1. ОЧИЩЕННЫЙ, СТРУКТУРИРОВАННЫЙ ТЕКСТ (Готов для LLM) ---")
    print(pure_text)
    print("\n--- КОЛИЧЕСТВО СИМВОЛОВ В ЧИСТОМ ТЕКСТЕ: {} ---".format(len(pure_text)))
    
    print("\n--- 2. НАЙДЕННЫЕ PDF-ДОКУМЕНТЫ (ССЫЛКИ) ---")
    if pdf_links:
        for title, link in pdf_links:
            print(f"- {title}: {link}")
    else:
        print("PDF-ссылки на странице не найдены.")

# --- ПРИМЕР ИСПОЛЬЗОВАНИЯ ---
if __name__ == "__main__":
    # URL с таблицами программ обучения
    test_url = "https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/"
    
    # URL с обычным текстовым контентом
    # test_url = "https://priem.stankin.ru/obshchezhitie/" 
    
    parse_stankin_page(test_url)

INFO: Попытка загрузки URL: https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/
INFO: SUCCESS: Страница успешно загружена.



🤖 РЕЗУЛЬТАТ ПАРСИНГА СТРАНИЦЫ: https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/

--- 1. ОЧИЩЕННЫЙ, СТРУКТУРИРОВАННЫЙ ТЕКСТ (Готов для LLM) ---
Программы обучения бакалавриата, специалитета 01.03.04 Прикладная математика Форма обучения: очная Стоимость обучения для граждан РФ: 164 000 руб/сем Стоимость обучения для иностранных граждан: 172 500 руб/сем Предметы: Р (min 40 баллов) + М (min 40 баллов) + И (min 44 балла)/Ф (min 39 баллов) Количество мест: Бюджетные (с учетом квот) Платные для граждан РФ Платные для иностранных граждан — 20 5 Отдельная квота: — мест Особая квота: — мест Целевая квота: — мест Проходные баллы: 2025 2024 2023 2022 2021 — — — — — Подробнее о направлении 09.03.01 Информатика и вычислительная техника Форма обучения: очная Стоимость обучения для граждан РФ: 182 100 руб/сем Стоимость обучения для иностранных граждан: 190 000 руб/сем Предметы: Р (min 40 баллов) + М (min 40 баллов) + И (min 44 балла) Количество мест: Бюджетные (с учетом квот) Плат

In [28]:
import requests
import logging
import re
import json
from typing import Optional, Tuple, List, Dict
from bs4 import BeautifulSoup, Comment
from urllib.parse import urljoin

# Настройка базового логирования (оставим для полноты)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# =========================================================
# 1. ЗАГРУЗКА HTML-КОДА (Оставлено без изменений)
# =========================================================

def fetch_html_content(url: str) -> Optional[str]:
    """Скачивает HTML-контент страницы."""
    try:
        logging.info(f"Попытка загрузки URL: {url}")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, timeout=15, headers=headers)
        response.raise_for_status()
        response.encoding = response.apparent_encoding
        logging.info("SUCCESS: Страница успешно загружена.")
        return response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка запроса к {url}: {e}")
        return None

# =========================================================
# 2. ОЧИСТКА И ПАРСИНГ (Оставлено без изменений)
# =========================================================

def clean_html_content_and_extract_links(html_content: str, url: str) -> Tuple[str, List[Tuple[str, str]]]:
    """
    Агрессивно очищает HTML, удаляет сквозные блоки и извлекает
    структурированный чистый текст, попутно собирая PDF-ссылки.
    """
    soup = BeautifulSoup(html_content, 'lxml')
    pdf_links = []
    
    # 1. Удаление служебных, нетекстовых тегов и комментариев
    REMOVAL_TAGS = ['script', 'style', 'noscript', 'iframe', 'meta', 'link', 'br']
    for element in soup(REMOVAL_TAGS):
        element.decompose()
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()
    
    # 2. Удаление сквозных и нерелевантных блоков
    elements_to_remove = [
        soup.find('div', class_='block-0-menu-16'),
        soup.find('nav', id='menu'),
        soup.find('div', id='n'),
        soup.find('header', class_='landing-header'),
        soup.find('div', class_='landing-footer'),
        soup.find('style', type='text/css'),
    ]

    for element in elements_to_remove:
        if element:
            element.decompose()
            
    # 3. Извлечение PDF-ссылок
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.lower().endswith('.pdf') or (link.text and 'pdf' in link.text.lower()):
            title = link.text.strip() or f"PDF-{len(pdf_links) + 1}"
            absolute_url = urljoin(url, href)
            pdf_links.append((title, absolute_url))
            link.decompose()
            
    # 4. Извлечение текста из основного контейнера
    main_content_tag = soup.find('div', class_='landing-main')
    if not main_content_tag:
        main_content_tag = soup.body if soup.body else soup

    for tag_name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'section', 'li', 'hr', 'table']:
        for tag in main_content_tag.find_all(tag_name):
            tag.append('\n')

    pure_text = main_content_tag.get_text(separator=' ', strip=True)

    # 5. Финальная чистка текста
    pure_text = re.sub(r'[\s]{2,}', '\n', pure_text)
    
    return pure_text.strip(), pdf_links


# =========================================================
# 3. ФУНКЦИИ СТРУКТУРИРОВАНИЯ ДАННЫХ
# =========================================================

def get_program_level(code: str) -> str:
    """Определяет уровень образования по коду (XX.03.XX -> Бакалавриат)."""
    if re.match(r'\d{2}\.03\.\d{2}', code):
        return "Бакалавриат"
    if re.match(r'\d{2}\.05\.\d{2}', code):
        return "Специалитет"
    return "Другое"

def extract_structured_data(full_text: str) -> List[Dict]:
    """
    Извлекает структурированные данные, ища каждый полный блок программы.
    """
    # Ищем все блоки, которые начинаются с кода направления и заканчиваются
    # перед следующим кодом или концом текста.
    pre_filter_match = re.split(r'Наименование направления подготовки', full_text, 1, re.DOTALL)
    full_text_filtered = pre_filter_match[0].strip()
    
    # Теперь ищем блоки только в отфильтрованном тексте
    program_blocks = re.findall(
        r'(\d{2}\.\d{2}\.\d{2}.*?)(?=\d{2}\.\d{2}\.\d{2}|$)', 
        full_text_filtered, re.DOTALL
    )
    
    structured_programs = []
    
    for block_content in program_blocks:
        if not block_content: continue

        # 1. Извлечение Кода и Названия
        # Код направления (XX.XX.XX)
        code_match = re.search(r'^(\d{2}\.\d{2}\.\d{2}(?:\.\d{2})?)', block_content)
        program_code = code_match.group(1) if code_match else "N/A"
        
        # Название остается без изменений, так как теперь мы пропускаем полный код.
        name_match = re.search(r'^\d{2}\.\d{2}\.\d{2}(?:\.\d{2})?\s*(.*?)(?=Форма обучения)', block_content, re.DOTALL)
        program_name = name_match.group(1).strip() if name_match else "N/A"
        
        # 2. Извлечение Формы обучения
        form_match = re.search(r'Форма обучения:\s*([^\s]+)', block_content)
        form_обучения = form_match.group(1).strip() if form_match else "N/A"

        # 3. Извлечение Предметов (Р + М + И/Ф)
        # Ищем все, что идет после "Предметы:" и до "Количество мест:" или "Стоимость обучения..."
        subjects_match = re.search(
            r'Предметы:\s*(.*?)(?=\s*Количество мест:|Стоимость обучения)', 
            block_content, re.DOTALL
        )
            
        subjects_str = subjects_match.group(1).strip() if subjects_match else "N/A"
        
        # Финальная очистка: удаляем все внутри скобок (min баллы)
        # Улучшенная логика для сохранения слеша /
        subjects_clean_str = re.sub(r'\s*\([^)]*\)', '', subjects_str)
        # Заменяем все последовательности +/ на + и удаляем лишние пробелы вокруг + и /
        subjects_clean = re.sub(r'\s*([+/])\s*', r' \1 ', subjects_clean_str).replace('  ', ' ').strip()
        
        final_subjects_value = subjects_clean


        # 4. Извлечение Стоимости (ищем цифры, игнорируя валюту и период)
        
        # Шаблон 1: Ищем "Стоимость обучения [возможное_слово] граждан РФ:"
        # (?:[^\s]+\s*)? - не захватывающая группа: 0 или 1 слово (для/лдя/и т.д.) + пробел.
        cost_rf_match_explicit = re.search(r'Стоимость обучения\s*(?:[^\s]+\s*)?граждан РФ:\s*(\d+\s*\d+)', block_content)
        стоимость_рф = cost_rf_match_explicit.group(1).replace(' ', '') if cost_rf_match_explicit else "N/A"
        
        # Шаблон 2: Стоимость обучения: 182 100 (как запасной вариант для РФ, если нет "граждан РФ")
        if стоимость_рф == "N/A":
            cost_rf_match_general = re.search(r'Стоимость обучения:\s*(\d+\s*\d+)', block_content)
            if cost_rf_match_general:
                стоимость_рф = cost_rf_match_general.group(1).replace(' ', '')

        # Стоимость для иностранных граждан
        cost_inostr_match = re.search(r'Стоимость обучения для иностранных граждан:\s*(\d+\s*\d+)', block_content)
        стоимость_иностр = cost_inostr_match.group(1).replace(' ', '') if cost_inostr_match else "N/A"
            
        # 5. Извлечение Проходных баллов за все года
        scores_match = re.search(
            r'Проходные баллы:\s*(?:2025|—)\s*(?:2024|—)\s*(?:2023|—)\s*(?:2022|—)\s*(?:2021|—)\s*((\d+|—)\s+(\d+|—)\s+(\d+|—)\s+(\d+|—)\s+(\d+|—))',
            block_content, re.DOTALL | re.IGNORECASE
        )
        
        scores = ["N/A"] * 5
        if scores_match:
            score_values = re.findall(r'(\d+|—)', scores_match.group(1))
            if len(score_values) >= 5:
                # 2025 | 2024 | 2023 | 2022 | 2021
                scores = [s if s != '—' else "N/A" for s in score_values[:5]]


        # 6. Извлечение Мест (Бюджетные, Платные РФ, Платные Иностр)
        quota_match = re.search(
            r'Бюджетные \(с учетом квот\)\s*Платные для граждан РФ\s*Платные для иностранных граждан\s*((\d+|—)\s+(\d+|—)\s+(\d+|—))', 
            block_content, re.DOTALL
        )

        places = ["0"] * 3
        if quota_match:
            place_values = re.findall(r'(\d+|—)', quota_match.group(1))
            if len(place_values) >= 3:
                places = [p if p != '—' else "0" for p in place_values[:3]]
        
        
        # 7. Извлечение Квот (Отдельная, Особая, Целевая)
        separate_quota_match = re.search(r'Отдельная квота:\s*(\d+|—)\s*мест', block_content)
        special_quota_match = re.search(r'Особая квота:\s*(\d+|—)\s*мест', block_content)
        target_quota_match = re.search(r'Целевая квота:\s*(\d+|—)\s*мест', block_content)
        
        separate_quota_value = separate_quota_match.group(1) if separate_quota_match and separate_quota_match.group(1) != '—' else "0"
        special_quota_value = special_quota_match.group(1) if special_quota_match and special_quota_match.group(1) != '—' else "0"
        target_quota_value = target_quota_match.group(1) if target_quota_match and target_quota_match.group(1) != '—' else "0"

        
        # ИНИЦИАЛИЗАЦИЯ СЛОВАРЯ program
        program = {
            "Код": program_code,
            "Направление": program_name.strip(),
            "Форма_обучения": form_обучения,
            "Предметы": final_subjects_value,
            "Стоимость_РФ": стоимость_рф,
            "Стоимость_Иностр": стоимость_иностр,
            
            # Проходные баллы
            "Проходной_2025": scores[0],
            "Проходной_2024": scores[1],
            "Проходной_2023": scores[2],
            "Проходной_2022": scores[3],
            "Проходной_2021": scores[4],
            
            # Места
            "Места_Бюджет": places[0],
            "Места_Платные_РФ": places[1],
            "Места_Платные_Иностр": places[2],
            
            # Квоты
            "Квота_Отдельная": separate_quota_value,
            "Квота_Особая": special_quota_value,
            "Квота_Целевая": target_quota_value,
        }
        
        # Добавляем для справки
        program["Уровень"] = get_program_level(program_code)
        
        structured_programs.append(program)
        
    return structured_programs

# =========================================================
# 4. ФИНАЛЬНЫЙ ЗАПУСК (ВЫВОД JSON)
# =========================================================

def parse_stankin_page(url: str):
    """Основная функция для парсинга страницы, возвращающая структурированные данные."""
    html_code = fetch_html_content(url)

    if not html_code:
        logging.error("Парсинг остановлен, не удалось загрузить HTML.")
        return

    # Шаг 1: Извлекаем чистый текст и список PDF-ссылок
    pure_text, pdf_links = clean_html_content_and_extract_links(html_code, url)
    
    # Шаг 2: Структурируем чистый текст в JSON-подобный формат
    structured_data = extract_structured_data(pure_text)
    
    print("\n" + "="*70)
    print(f"🤖 РЕЗУЛЬТАТ ПАРСИНГА СТРАНИЦЫ: {url}")
    print("="*70)
    
    print("\n--- 1. СТРУКТУРИРОВАННЫЕ ДАННЫЕ (JSON-ФОРМАТ) ---")
    
    # Вывод в красивом JSON-формате
    print(json.dumps(structured_data, ensure_ascii=False, indent=4))
    print(f"\nИТОГО: Успешно извлечено {len(structured_data)} программ.")
    
    print("\n--- 2. НАЙДЕННЫЕ PDF-ДОКУМЕНТЫ (ССЫЛКИ) ---")
    if pdf_links:
        for title, link in pdf_links:
            print(f"- {title}: {link}")
    else:
        print("PDF-ссылки на странице не найдены.")

# --- ПРИМЕР ИСПОЛЬЗОВАНИЯ ---
if __name__ == "__main__":
    # URL с программами обучения
    test_url = "https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/"
    
    parse_stankin_page(test_url)

INFO: Попытка загрузки URL: https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/
INFO: SUCCESS: Страница успешно загружена.



🤖 РЕЗУЛЬТАТ ПАРСИНГА СТРАНИЦЫ: https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/

--- 1. СТРУКТУРИРОВАННЫЕ ДАННЫЕ (JSON-ФОРМАТ) ---
[
    {
        "Код": "01.03.04",
        "Направление": "Прикладная математика",
        "Форма_обучения": "очная",
        "Предметы": "Р + М + И / Ф",
        "Стоимость_РФ": "164000",
        "Стоимость_Иностр": "172500",
        "Проходной_2025": "N/A",
        "Проходной_2024": "N/A",
        "Проходной_2023": "N/A",
        "Проходной_2022": "N/A",
        "Проходной_2021": "N/A",
        "Места_Бюджет": "0",
        "Места_Платные_РФ": "20",
        "Места_Платные_Иностр": "5",
        "Квота_Отдельная": "0",
        "Квота_Особая": "0",
        "Квота_Целевая": "0",
        "Уровень": "Бакалавриат"
    },
    {
        "Код": "09.03.01",
        "Направление": "Информатика и вычислительная техника",
        "Форма_обучения": "очная",
        "Предметы": "Р + М + И",
        "Стоимость_РФ": "182100",
        "Стоимость_Иностр":

C:\Users\MainAdmUser\AppData\Local\Temp\ipykernel_13436\3493176182.py:109: DeprecationWarning: 'maxsplit' is passed as positional argument
  pre_filter_match = re.split(r'Наименование направления подготовки', full_text, 1, re.DOTALL)


In [29]:
import requests
import logging
import re
import json
from typing import Optional, List, Dict
from bs4 import BeautifulSoup, Comment

# Настройка базового логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# =========================================================
# 1. ЗАГРУЗКА HTML-КОДА
# =========================================================

def fetch_html_content(url: str) -> Optional[str]:
    """Скачивает HTML-контент страницы."""
    try:
        logging.info(f"Попытка загрузки URL: {url}")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, timeout=15, headers=headers)
        response.raise_for_status()
        response.encoding = response.apparent_encoding
        logging.info("SUCCESS: Страница успешно загружена.")
        return response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка запроса к {url}: {e}")
        return None

# =========================================================
# 2. ОЧИСТКА ТЕКСТА
# =========================================================

def clean_html_content(html_content: str) -> str:
    """
    Очищает HTML, удаляет лишние теги и возвращает чистый текст.
    """
    soup = BeautifulSoup(html_content, 'lxml')
    
    # 1. Удаление служебных, нетекстовых тегов и комментариев
    REMOVAL_TAGS = ['script', 'style', 'noscript', 'iframe', 'meta', 'link', 'br']
    for element in soup(REMOVAL_TAGS):
        element.decompose()
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()
    
    # 2. Удаление навигации и футеров
    elements_to_remove = [
        soup.find('div', class_='block-0-menu-16'),
        soup.find('nav', id='menu'),
        soup.find('div', id='n'),
        soup.find('header', class_='landing-header'),
        soup.find('div', class_='landing-footer'),
        soup.find('style', type='text/css'),
    ]

    for element in elements_to_remove:
        if element:
            element.decompose()
            
    # 3. Извлечение текста
    main_content_tag = soup.find('div', class_='landing-main')
    if not main_content_tag:
        main_content_tag = soup.body if soup.body else soup

    for tag_name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'section', 'li', 'hr', 'table']:
        for tag in main_content_tag.find_all(tag_name):
            tag.append('\n')

    pure_text = main_content_tag.get_text(separator=' ', strip=True)

    # 4. Финальная нормализация пробелов
    pure_text = re.sub(r'[\s]{2,}', '\n', pure_text)
    
    return pure_text.strip()


# =========================================================
# 3. ПАРСИНГ ДАННЫХ
# =========================================================

def get_program_level(code: str) -> str:
    """Определяет уровень образования по коду."""
    if re.match(r'\d{2}\.03\.\d{2}', code):
        return "Бакалавриат"
    if re.match(r'\d{2}\.05\.\d{2}', code):
        return "Специалитет"
    return "Другое"

def extract_structured_data(full_text: str) -> List[Dict]:
    """
    Парсит текст и извлекает данные по программам.
    """
    # 1. Отсекаем нижнюю таблицу (заголовки), которая ломает парсинг
    pre_filter_match = re.split(r'Наименование направления подготовки', full_text, 1, re.DOTALL)
    full_text_filtered = pre_filter_match[0].strip()
    
    # 2. Разбиваем текст на блоки программ
    program_blocks = re.findall(
        r'(\d{2}\.\d{2}\.\d{2}.*?)(?=\d{2}\.\d{2}\.\d{2}|$)', 
        full_text_filtered, re.DOTALL
    )
    
    structured_programs = []
    
    for block_content in program_blocks:
        if not block_content: 
            continue

        # Пропускаем блоки без ключевых слов (защита от мусора)
        if 'Форма обучения' not in block_content:
            continue

        # --- Код и Название ---
        # Код включает опциональный под-код (XX.XX.XX.XX)
        code_match = re.search(r'^(\d{2}\.\d{2}\.\d{2}(?:\.\d{2})?)', block_content)
        program_code = code_match.group(1) if code_match else "N/A"
        
        # Название: берем текст после кода до фразы "Форма обучения"
        name_match = re.search(r'^\d{2}\.\d{2}\.\d{2}(?:\.\d{2})?\s*(.*?)(?=Форма обучения)', block_content, re.DOTALL)
        program_name = name_match.group(1).strip() if name_match else "N/A"
        
        # --- Форма обучения ---
        form_match = re.search(r'Форма обучения:\s*([^\s]+)', block_content)
        study_form = form_match.group(1).strip() if form_match else "N/A"

        # --- Предметы ---
        subjects_match = re.search(
            r'Предметы:\s*(.*?)(?=\s*Количество мест:|Стоимость обучения)', 
            block_content, re.DOTALL
        )
        subjects_str = subjects_match.group(1).strip() if subjects_match else "N/A"
        
        # Очистка предметов (удаляем скобки с баллами, нормализуем разделители)
        subjects_clean_str = re.sub(r'\s*\([^)]*\)', '', subjects_str)
        subjects_final = re.sub(r'\s*([+/])\s*', r' \1 ', subjects_clean_str).replace('  ', ' ').strip()


        # --- Стоимость (РФ) ---
        # Ищем паттерн с защитой от опечаток в слове "для" (лдя/пропуск слова)
        cost_rf_match_explicit = re.search(r'Стоимость обучения\s*(?:[^\s]+\s*)?граждан РФ:\s*(\d+\s*\d+)', block_content)
        cost_rf = cost_rf_match_explicit.group(1).replace(' ', '') if cost_rf_match_explicit else "N/A"
        
        # Резервный поиск, если не нашли явного упоминания РФ
        if cost_rf == "N/A":
            cost_rf_match_general = re.search(r'Стоимость обучения:\s*(\d+\s*\d+)', block_content)
            if cost_rf_match_general:
                cost_rf = cost_rf_match_general.group(1).replace(' ', '')

        # --- Стоимость (Иностранцы) ---
        cost_foreign_match = re.search(r'Стоимость обучения для иностранных граждан:\s*(\d+\s*\d+)', block_content)
        cost_foreign = cost_foreign_match.group(1).replace(' ', '') if cost_foreign_match else "N/A"
            
        # --- Проходные баллы (История) ---
        scores_match = re.search(
            r'Проходные баллы:\s*(?:2025|—)\s*(?:2024|—)\s*(?:2023|—)\s*(?:2022|—)\s*(?:2021|—)\s*((\d+|—)\s+(\d+|—)\s+(\d+|—)\s+(\d+|—)\s+(\d+|—))',
            block_content, re.DOTALL | re.IGNORECASE
        )
        
        scores = ["N/A"] * 5
        if scores_match:
            score_values = re.findall(r'(\d+|—)', scores_match.group(1))
            if len(score_values) >= 5:
                # 2025 -> index 0, 2021 -> index 4
                scores = [s if s != '—' else "N/A" for s in score_values[:5]]


        # --- Количество мест ---
        quota_match = re.search(
            r'Бюджетные \(с учетом квот\)\s*Платные для граждан РФ\s*Платные для иностранных граждан\s*((\d+|—)\s+(\d+|—)\s+(\d+|—))', 
            block_content, re.DOTALL
        )

        places = ["0"] * 3
        if quota_match:
            place_values = re.findall(r'(\d+|—)', quota_match.group(1))
            if len(place_values) >= 3:
                places = [p if p != '—' else "0" for p in place_values[:3]]
        
        # --- Квоты ---
        separate_quota_match = re.search(r'Отдельная квота:\s*(\d+|—)\s*мест', block_content)
        special_quota_match = re.search(r'Особая квота:\s*(\d+|—)\s*мест', block_content)
        target_quota_match = re.search(r'Целевая квота:\s*(\d+|—)\s*мест', block_content)
        
        quota_separate = separate_quota_match.group(1) if separate_quota_match and separate_quota_match.group(1) != '—' else "0"
        quota_special = special_quota_match.group(1) if special_quota_match and special_quota_match.group(1) != '—' else "0"
        quota_target = target_quota_match.group(1) if target_quota_match and target_quota_match.group(1) != '—' else "0"

        
        # Сборка объекта
        program = {
            "Код": program_code,
            "Направление": program_name.strip(),
            "Форма_обучения": study_form,
            "Предметы": subjects_final,
            "Стоимость_РФ": cost_rf,
            "Стоимость_Иностр": cost_foreign,
            
            "Проходной_2025": scores[0],
            "Проходной_2024": scores[1],
            "Проходной_2023": scores[2],
            "Проходной_2022": scores[3],
            "Проходной_2021": scores[4],
            
            "Места_Бюджет": places[0],
            "Места_Платные_РФ": places[1],
            "Места_Платные_Иностр": places[2],
            
            "Квота_Отдельная": quota_separate,
            "Квота_Особая": quota_special,
            "Квота_Целевая": quota_target,
        }
        
        program["Уровень"] = get_program_level(program_code)
        
        structured_programs.append(program)
        
    return structured_programs

# =========================================================
# 4. ЗАПУСК
# =========================================================

def parse_stankin_page(url: str):
    """Основная точка входа."""
    html_code = fetch_html_content(url)

    if not html_code:
        logging.error("Не удалось получить контент страницы.")
        return

    pure_text = clean_html_content(html_code)
    structured_data = extract_structured_data(pure_text)
    
    print("\n" + "="*70)
    print(f"🤖 РЕЗУЛЬТАТ ПАРСИНГА: {url}")
    print("="*70)
    
    print(json.dumps(structured_data, ensure_ascii=False, indent=4))
    print(f"\nИТОГО: Успешно извлечено {len(structured_data)} программ.")

if __name__ == "__main__":
    test_url = "https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/"
    parse_stankin_page(test_url)

INFO: Попытка загрузки URL: https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/
INFO: SUCCESS: Страница успешно загружена.
C:\Users\MainAdmUser\AppData\Local\Temp\ipykernel_13436\3307126572.py:96: DeprecationWarning: 'maxsplit' is passed as positional argument
  pre_filter_match = re.split(r'Наименование направления подготовки', full_text, 1, re.DOTALL)



🤖 РЕЗУЛЬТАТ ПАРСИНГА: https://priem.stankin.ru/bakalavriatispetsialitet/training_programs/
[
    {
        "Код": "01.03.04",
        "Направление": "Прикладная математика",
        "Форма_обучения": "очная",
        "Предметы": "Р + М + И / Ф",
        "Стоимость_РФ": "164000",
        "Стоимость_Иностр": "172500",
        "Проходной_2025": "N/A",
        "Проходной_2024": "N/A",
        "Проходной_2023": "N/A",
        "Проходной_2022": "N/A",
        "Проходной_2021": "N/A",
        "Места_Бюджет": "0",
        "Места_Платные_РФ": "20",
        "Места_Платные_Иностр": "5",
        "Квота_Отдельная": "0",
        "Квота_Особая": "0",
        "Квота_Целевая": "0",
        "Уровень": "Бакалавриат"
    },
    {
        "Код": "09.03.01",
        "Направление": "Информатика и вычислительная техника",
        "Форма_обучения": "очная",
        "Предметы": "Р + М + И",
        "Стоимость_РФ": "182100",
        "Стоимость_Иностр": "190000",
        "Проходной_2025": "248",
        "Проходн